In [14]:
from itertools import product

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from download_data import get_price, get_mpd, get_fx, get_commodity, get_bond, get_inflation
from data_analysis.explore_data import (
    calc_rolling_stat,
    merge_prob_stat,
    rename_stat_df,
    eval_large_change_prob,
    plot_change_scatter,
    prep_regression_stat,
    compare_regressions,
    compare_regression_eval
)

In [15]:
mpd = get_mpd()
start_date, end_date = mpd["idt"].min(), mpd["idt"].max()

equity = get_price(use_cache=True, tickers=["BAC", "C", "IYR", "SPY"], 
                  start_date=start_date, 
                  end_date=end_date, 
                  )

fx = get_fx(use_cache=True, tickers=["GBP", "EUR", "JPY"],
            start_date=start_date,
            end_date=end_date,
            )

commodity = get_commodity(use_cache=True, 
                          start_date=start_date, 
                          end_date=end_date,)

inflation = get_inflation()
bond = get_bond(use_cache=True)

In [16]:
mpd["market"].unique()

array(['bac', 'citi', 'corn', 'euro', 'gold', 'infl1y', 'infl2y',
       'infl5y', 'iyr', 'LR3y3m', 'LR5y3m', 'oil', 'pound', 'silver',
       'soybns', 'sp12m', 'sp6m', 'tr10yr', 'tr5yr', 'wheat', 'yen'],
      dtype=object)

In [17]:
stat = calc_rolling_stat(
    pd.concat([fx, equity, commodity, bond, inflation], axis=1), 
    month=6
)
stat


KeyboardInterrupt



In [ ]:
market_list = ["bac", "citi", "iyr", "sp6m", "sp12m", "euro", "pound", "yen"] + ['silver','corn', 'soybns','gold','wheat'] + ["infl1y", "infl2y", "infl5y", "tr5yr", "tr10yr"] 
ticker_list = ["BAC", "C", "IYR", "SPY", "SPY", "GBP", "EUR", "JPY"] + ['silver','corn1', 'soybean','gold','wheat'] + ['EXPINF1YR', 'EXPINF2YR', 'EXPINF5YR', '5YR_Price', '10YR_Price']
stat_rename = rename_stat_df(stat, dict(zip(market_list, ticker_list)))

In [ ]:
df = merge_prob_stat(stat_rename, mpd)

In [ ]:
import numpy as np
from scipy.stats import chi2

def kupiec_pof_test(total_observations, exceedances, confidence_level: float = 0.99):
    """
    Calculate the Kupiec POF test statistic.

    Parameters:
    total_observations (int): Total number of observations in the dataset.
    exceedances (int): Number of observed exceedances (VaR failures).
    confidence_level (float): The confidence level used for the VaR calculation (e.g., 0.99 for 99%).

    Returns:
    float: The Kupiec POF test statistic.
    float: The p-value from the chi-squared distribution.
    """
    # Probability of exceedance
    p = 1 - confidence_level

    # Expected number of exceedances
    expected_exceedances = total_observations * p

    # Actual proportion of exceedances
    proportion_exceedances = exceedances / total_observations

    # Kupiec test statistic
    LR_POF = -2 * np.log(((1 - p) ** (total_observations - exceedances)) * (p ** exceedances)) + 2 * np.log(((1 - proportion_exceedances) ** (total_observations - exceedances)) * (proportion_exceedances ** exceedances))

    # Calculate the p-value
    p_value = 1 - chi2.cdf(LR_POF, 1)

    return LR_POF, p_value

# Example usage
total_observations = 250  # Total number of days in the dataset
exceedances = 5  # Number of days when the loss exceeded the VaR
confidence_level = 0.99  # 99% confidence level

LR_POF, p_value = kupiec_pof_test(total_observations, exceedances, confidence_level)

print(f"Kupiec POF Test Statistic: {LR_POF}")
print(f"P-Value: {p_value}")

In [24]:
def calc_hit_rate(df: pd.DataFrame, idx: list[str] = market_list):
    p_col = ["p10", "p50", "p90"]
    for c in p_col:
        df.loc[:, f"{c}_hit"] = df[c] < df["ret"]
        
    hit_ratio = {}
    k_test = {}
    k_pvalue = {}
    for name, g in df.filter(regex="_hit$").groupby("market"):
        hit_ratio[name] = g.mean()
        k_test[name], k_pvalue[name] = kupiec_pof_test(len(g), g.sum())
    
    
    df = pd.concat([pd.DataFrame(hit_ratio), 
                    pd.DataFrame(k_test), 
                    pd.DataFrame(k_pvalue)], keys=["hit_ratio", "kupiec_stat", "kupiec_pvalue"]).T
    return df.reindex(idx)

calc_hit_rate(df)

/Users/claircui/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/claircui/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/claircui/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/claircui/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/claircui/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/claircui/anaconda3/lib/python3.11/site-packages/panda

hit_ratio                     kupiec_stat                          \
         p10_hit   p50_hit   p90_hit     p10_hit     p50_hit     p90_hit   
bac     0.877076  0.586379  0.209302         inf         inf  552.381680   
citi    0.901993  0.528239  0.162791         inf         inf  377.844635   
iyr     0.941924  0.627949  0.161525         inf         inf  341.714277   
sp6m    0.977712  0.619614  0.020802         inf         inf    6.049118   
sp12m   0.991085  0.566122  0.000000         inf         inf   13.527752   
euro    0.903169  0.503521  0.213028         inf         inf  535.054949   
pound   0.943662  0.552817  0.160211         inf         inf  347.866225   
yen     0.917254  0.577465  0.198944         inf         inf  483.124634   
silver  0.957935  0.443595  0.072658         inf         inf   87.300521   
corn    0.967619  0.481905  0.043810         inf         inf   33.067444   
soybns  0.960076  0.513308  0.076046         inf         inf   95.190804   
gold    0.975285  0.587452  0.076046         inf         inf   95.190804   
wheat   0.986692  0.528517  0.041825         inf         inf   30.024097   
infl1y  0.452436  0.452436  0.450116         inf         inf         inf   
infl2y  0.570175  0.342105  0.318713         inf  642.715812  580.497771   
infl5y  0.407738  0.385417  0.336310         inf         inf         inf   
tr5yr   0.856540  0.453586  0.158228         inf         inf  284.785583   
tr10yr  0.815261  0.451807  0.164659         inf         inf  318.084206   

       kupiec_pvalue                     
                   0    1             2  
bac              0.0  0.0  0.000000e+00  
citi             0.0  0.0  0.000000e+00  
iyr              0.0  0.0  0.000000e+00  
sp6m             0.0  0.0  1.391325e-02  
sp12m            0.0  0.0  2.350614e-04  
euro             0.0  0.0  0.000000e+00  
pound            0.0  0.0  0.000000e+00  
yen              0.0  0.0  0.000000e+00  
silver           0.0  0.0  0.000000e+00  
corn             0.0  0.0  8.901680e-09  
soybns           0.0  0.0  0.000000e+00  
gold             0.0  0.0  0.000000e+00  
wheat            0.0  0.0  4.267105e-08  
infl1y           0.0  0.0  0.000000e+00  
infl2y           0.0  0.0  0.000000e+00  
infl5y           0.0  0.0  0.000000e+00  
tr5yr            0.0  0.0  0.000000e+00  
tr10yr           0.0  0.0  0.000000e+00

In [23]:
df.index.get_level_values("idt").unique()

DatetimeIndex(['2010-01-15', '2010-01-29', '2010-02-12', '2010-02-26',
               '2010-03-15', '2010-03-31', '2010-04-15', '2010-04-30',
               '2010-05-14', '2010-05-28',
               ...
               '2009-07-15', '2009-07-31', '2009-08-14', '2009-08-31',
               '2009-09-15', '2009-09-30', '2009-10-30', '2009-11-13',
               '2009-11-30', '2009-12-15'],
              dtype='datetime64[ns]', name='idt', length=877, freq=None)